In [1]:
import numpy as np
import pandas as pd

# Problem 1
Given: Three positive integers k, m, and n, representing a population containing k+m+n organisms: k individuals are homozygous dominant for a factor, m are heterozygous, and n are homozygous recessive.

Return: The probability that two randomly selected mating organisms will produce an individual possessing a dominant allele (and thus displaying the dominant phenotype). Assume that any two organisms can mate.


In [17]:
def all_permutations(s1, s2, sort=False):
    x = list()
    for a in s1:
        for b in s2:
            s = a + b
            if sort:
                s = ''.join(sorted(s))
            x.append(s)
    return x

def count(x, normalize=False):
    d = dict()
    for v in x:
        if v not in d:
            d[v] = 0
        d[v] += 1
    if normalize:
        the_sum = np.sum(list(d.values()))
        for k in d.keys():
            d[k] /= the_sum
    return d

def punette(genotype1, genotype2):
    perms = all_permutations(genotype1, genotype2, sort=True)
    return count(perms, normalize=True)

def selection_prob(k, m, n, progeny_genotypes):
    p = np.array([k, m, n]).astype('float')
    p /= p.sum()
    genotypes = ['AA', 'Aa', 'aa']
    p_total = 0.0
    for a in range(3):
        for b in range(3):
            p_combo = p[a]*p[b]
            sq = punette(genotypes[a], genotypes[b])
            for pg in progeny_genotypes:
                p_progeny_genotype = sq.get(pg, 0.0)
                p_total += p_combo*p_progeny_genotype
                print('{}/{} ({:0.2f}/{:0.2f}): pg={}, p(pg)={:0.2f}, p_total={:0.2f}'.\
                     format(genotypes[a], genotypes[b], p[a], p[b], pg, p_progeny_genotype, p_total))
    return p_total
    
selection_prob(2, 2, 2, ['AA', 'Aa'])

AA/AA (0.33/0.33): pg=AA, p(pg)=1.00, p_total=0.11
AA/AA (0.33/0.33): pg=Aa, p(pg)=0.00, p_total=0.11
AA/Aa (0.33/0.33): pg=AA, p(pg)=0.50, p_total=0.17
AA/Aa (0.33/0.33): pg=Aa, p(pg)=0.50, p_total=0.22
AA/aa (0.33/0.33): pg=AA, p(pg)=0.00, p_total=0.22
AA/aa (0.33/0.33): pg=Aa, p(pg)=1.00, p_total=0.33
Aa/AA (0.33/0.33): pg=AA, p(pg)=0.50, p_total=0.39
Aa/AA (0.33/0.33): pg=Aa, p(pg)=0.50, p_total=0.44
Aa/Aa (0.33/0.33): pg=AA, p(pg)=0.25, p_total=0.47
Aa/Aa (0.33/0.33): pg=Aa, p(pg)=0.50, p_total=0.53
Aa/aa (0.33/0.33): pg=AA, p(pg)=0.00, p_total=0.53
Aa/aa (0.33/0.33): pg=Aa, p(pg)=0.50, p_total=0.58
aa/AA (0.33/0.33): pg=AA, p(pg)=0.00, p_total=0.58
aa/AA (0.33/0.33): pg=Aa, p(pg)=1.00, p_total=0.69
aa/Aa (0.33/0.33): pg=AA, p(pg)=0.00, p_total=0.69
aa/Aa (0.33/0.33): pg=Aa, p(pg)=0.50, p_total=0.75
aa/aa (0.33/0.33): pg=AA, p(pg)=0.00, p_total=0.75
aa/aa (0.33/0.33): pg=Aa, p(pg)=0.00, p_total=0.75


0.75

# Problem 2

Given: Two DNA strings s and t (each of length at most 1 kbp).

Return: All locations of t as a substring of s.

In [19]:
def all_substrings(s, t):
    locs = list()
    tlen = len(t)
    for k in range(len(s)-tlen):
        if s[k:(k+tlen)] == t:
            locs.append(k)
    return locs
                   
all_substrings('GATATATGCATATACTT', 'ATAT')

[1, 3, 9]

# Problem 3

Given: A collection of k (k≤100) DNA strings of length at most 1 kbp each in FASTA format.

Return: A longest common substring of the collection. (If multiple solutions exist, you may return any single solution.)

In [45]:
def alignment_matrix(s1, s2):
    A = np.zeros([len(s1), len(s2)])
    for k in range(len(s1)):
        for j in range(len(s2)):
            A[k, j] = int(s1[k] == s2[j])
    return A

def follow_diagonal(A, s1, s2, kk, jj, seq=list(), hits=dict()):
    #print('[fd] ({}, {}), seq={}'.format(kk, jj, ''.join(seq)))
    if kk >= A.shape[0] or jj >= A.shape[1]:
        return seq
    if (kk, jj) in hits:
        return seq
    if A[kk, jj] == 1:
        seq.append(s1[kk])
        hits[(kk, jj)] = True
        return follow_diagonal(A, s1, s2, kk+1, jj+1, seq, hits)
    else:
        return seq

def local_matches(s1, s2):
    A = alignment_matrix(s1, s2)
    #print(A)
    matches = dict()
    hits = dict()
    for k in range(len(s1)):
        for j in range(len(s2)):
            if (k, j) not in hits and A[k, j] == 1:
                seq = follow_diagonal(A, s1, s2, k, j, seq=list(), hits=hits)
                #print('({}, {}): seq={}'.format(k, j, seq))
                if len(seq) > 0:
                    matches[''.join(seq)] = True
    return list(matches.keys())

def largest_common_substring(strs):
    assert len(strs) > 1
    last_matches = local_matches(strs[0], strs[1])
    print('k=-1, last_matches={}'.format(','.join(last_matches)))
    for k in range(1, len(strs)-1):
        m = local_matches(strs[k], strs[k+1])
        print('k={}, m={}'.format(k, ','.join(m)))
        matches_left = list()
        for ii in range(len(last_matches)):
            for jj in range(len(m)):
                mm = local_matches(last_matches[ii], m[jj])
                if mm is not None and len(mm) > 0:
                    matches_left.extend(mm)
        last_matches = np.unique(matches_left)
    if len(last_matches) == 0:
        return None
    match_size = [len(x) for x in last_matches]
    i = np.argsort(match_size)[0]
    return last_matches[i]

#local_matches('ABCDEFGH', 'BCEFGH')
largest_common_substring(['ACDEFGH', 'BCEFGH', 'XXXXYBCOP'])

k=-1, last_matches=C,EFGH
k=1, m=BC


'C'

# Problem 4
Given: Two protein strings s and t in FASTA format (each having length at most 1000 aa).

Return: A maximum alignment score along with substrings r and u of s and t, respectively, which produce this maximum alignment score (multiple solutions may exist, in which case you may output any one). 

In [92]:

def PAM250():
    s = """A  C  D  E  F  G  H  I  K  L  M  N  P  Q  R  S  T  V  W  Y
    A  2 -2  0  0 -3  1 -1 -1 -1 -2 -1  0  1  0 -2  1  1  0 -6 -3
    C -2 12 -5 -5 -4 -3 -3 -2 -5 -6 -5 -4 -3 -5 -4  0 -2 -2 -8  0
    D  0 -5  4  3 -6  1  1 -2  0 -4 -3  2 -1  2 -1  0  0 -2 -7 -4
    E  0 -5  3  4 -5  0  1 -2  0 -3 -2  1 -1  2 -1  0  0 -2 -7 -4
    F -3 -4 -6 -5  9 -5 -2  1 -5  2  0 -3 -5 -5 -4 -3 -3 -1  0  7
    G  1 -3  1  0 -5  5 -2 -3 -2 -4 -3  0  0 -1 -3  1  0 -1 -7 -5
    H -1 -3  1  1 -2 -2  6 -2  0 -2 -2  2  0  3  2 -1 -1 -2 -3  0
    I -1 -2 -2 -2  1 -3 -2  5 -2  2  2 -2 -2 -2 -2 -1  0  4 -5 -1
    K -1 -5  0  0 -5 -2  0 -2  5 -3  0  1 -1  1  3  0  0 -2 -3 -4
    L -2 -6 -4 -3  2 -4 -2  2 -3  6  4 -3 -3 -2 -3 -3 -2  2 -2 -1
    M -1 -5 -3 -2  0 -3 -2  2  0  4  6 -2 -2 -1  0 -2 -1  2 -4 -2
    N  0 -4  2  1 -3  0  2 -2  1 -3 -2  2  0  1  0  1  0 -2 -4 -2
    P  1 -3 -1 -1 -5  0  0 -2 -1 -3 -2  0  6  0  0  1  0 -1 -6 -5
    Q  0 -5  2  2 -5 -1  3 -2  1 -2 -1  1  0  4  1 -1 -1 -2 -5 -4
    R -2 -4 -1 -1 -4 -3  2 -2  3 -3  0  0  0  1  6  0 -1 -2  2 -4
    S  1  0  0  0 -3  1 -1 -1  0 -3 -2  1  1 -1  0  2  1 -1 -2 -3
    T  1 -2  0  0 -3  0 -1  0  0 -2 -1  0  0 -1 -1  1  3  0 -5 -3
    V  0 -2 -2 -2 -1 -1 -2  4 -2  2  2 -2 -1 -2 -2 -1  0  4 -6 -2
    W -6 -8 -7 -7  0 -7 -3 -5 -3 -2 -4 -4 -6 -5  2 -2 -5 -6 17  0
    Y -3  0 -4 -4  7 -5  0 -1 -4 -1 -2 -2 -5 -4 -4 -3 -3 -2  0 10"""
    lns = s.split('\n')
    idx = lns[0].split()
    S = list()
    for k in range(1, len(lns)):
        x = [int(y) for y in lns[k][7:].split()]
        S.append(x)
    S = np.array(S)
    assert len(idx) == S.shape[0]
    return idx, S

def score_matrix(seq1, seq2, gap_penalty=10):
    protein_index, sub_matrix = PAM250()
    
    H = np.zeros([len(seq1)+1, len(seq2)+1])
    for k in range(1, len(seq1)):
        for j in range(1, len(seq2)):
            scores = [0]
            i1 = protein_index.index(seq1[k])
            i2 = protein_index.index(seq2[j])
            scores.append(H[k-1, j-1] + sub_matrix[i1, i2])
            
            row_scores = H[:k, j] - np.flip(np.arange(1, k+1))*gap_penalty
            scores.append(np.max(row_scores))
            
            col_scores = H[k, :j] - np.flip(np.arange(1, j+1))*gap_penalty
            scores.append(np.max(col_scores))
            
            H[k, j] = np.max(scores)
    return H

def local_alignment(seq1, seq2, gap_penalty=10):
    H = score_matrix(seq1, seq2, gap_penalty=gap_penalty)
    
    def argmax(X): return np.unravel_index(X.argmax(), X.shape)

    matched_sequence = list()
    # start at maximum element
    row, col = argmax(H)
    letter1, letter2 = seq1[row], seq2[col]
    while H[row, col] > 0:
        print('({}, {}) H={}, ({}, {})'.format(row, col, H[row, col], letter1, letter2))
        matched_sequence.append((letter1, letter2))
        # identify best next step: 0 for left, 1 for upper left, 2 for up
        next_scores = [H[row, col-1], H[row-1, col-1], H[row-1, col]]
        next_step = np.argmax(next_scores)
        if next_step == 1:
            row, col = row-1, col-1
            letter1, letter2 = seq1[row-1], seq2[col-1]
        elif next_step == 0:
            row, col = row, col-1
            letter1, letter2 = '-',seq2[col-1]
        elif next_step == 2:
            row, col = row-1, col
            letter1, letter2 = seq1[row-1], '-'
    m1 = ''.join([x[0] for x in matched_sequence[::-1]])
    m2 = ''.join([x[1] for x in matched_sequence[::-1]])
    return m1, m2
    
def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")
        
seq1 = 'MEANLYPRTEINSTRING'
seq2 = 'PLEASANTLYEINSTEIN'

H = score_matrix(seq1, seq2)
matprint(H)
#max_row, max_col = argmax(H)
m1, m2 = local_alignment(seq1, seq2)
print(m1)
print(m2)

0  0  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0  0  
0  0  4  0  0  0  1  0  0   0   4   0   1   0   0   4   0   1  0  
0  0  0  6  1  2  0  2  0   0   0   3   0   2   1   0   3   0  0  
0  0  1  0  7  1  4  0  0   0   1   0   5   1   2   2   0   5  0  
0  6  0  3  0  9  0  2  6   0   0   3   0   2   0   0   4   0  0  
0  0  2  3  0  3  7  0  1  16   6   0   1   0   0   0   0   2  0  
0  0  0  3  4  1  3  7  0   6  15   5   0   2   0   0   0   0  0  
0  0  0  2  3  6  1  2  4   0   5  13   5   0   1   0   0   0  0  
0  0  0  1  3  4  6  4  0   1   0   5  13   6   3   1   0   0  0  
0  0  4  0  1  3  5  6  1   0   5   0   6  13   6   7   0   1  0  
0  2  0  5  0  2  1  5  8   0   0  10   0   5  13   4  12   2  0  
0  0  3  0  6  0  4  1  2   6   1   0  12   2   5  14   4  14  0  
0  0  0  4  2  7  1  5  0   0   6   0   2  14   4   5  13   5  0  
0  0  0  1  5  3  7  4  3   0   0   6   0   4  17   7   5  13  0  
0  0  0  2  1  7  3  6  1   0   0   0   6   0   7  16   6   5 

# Problem 5

Make a function that produces all k-mers for a given k and four-letter alphabet A,C,T,G.

In [109]:
def kmers(k, alphabet=['A', 'C', 'T', 'G']):
    if k == 0:
        return list()
    
    klist = list()
    for j in range(len(alphabet)):
        base_str = alphabet[j]
        #print('base_str={}, j={}'.format(base_str, j))
        if k == 1:
            klist.append(base_str[0])
        else:
            for s in kmers(k-1, alphabet):
                pstr = base_str + s
                #print('pstr=',pstr)
                klist.append(pstr)
    #print('klist={}'.format(','.join(klist)))            
    return klist
        
kmers(2)

['AA',
 'AC',
 'AT',
 'AG',
 'CA',
 'CC',
 'CT',
 'CG',
 'TA',
 'TC',
 'TT',
 'TG',
 'GA',
 'GC',
 'GT',
 'GG']

# Problem 6

Suppose in a coin toss experiment, 11 heads are observed out of 26 tosses. Is the coin biased?

In [115]:
def factorial(n):
    prod = 1
    for k in range(1, n+1):
        prod *= k
    return prod

def binom_coeff(n, k):
    return factorial(n) / (factorial(k)*factorial(n-k))

def binom_prob(n, k, p):
    return binom_coeff(n, k) * (p**k) * ((1-p)**(n-k))

print('P(11 heads) with p=0.5: {:0.2f}'.format(binom_prob(26, 11, 0.5)))

p_at_least_11 = 0.
for k in range(11, 27):
    p_at_least_11 += binom_prob(26, k, 0.5)
print('P(>=11 heads) with p=0.5: {:0.2f}'.format(p_at_least_11))

p_less_than_11 = 0.
for k in range(12):
    p_less_than_11 += binom_prob(26, k, 0.5)
print('P(<=11 heads) with p=0.5: {:0.2f}'.format(p_less_than_11))


P(11 heads) with p=0.5: 0.12
P(>=11 heads) with p=0.5: 0.84
P(<=11 heads) with p=0.5: 0.28
